In [0]:
%sql
CREATE TABLE students
  (id INT, name STRING, value DOUBLE);

In [0]:
%sql
CREATE TABLE IF NOT EXISTS students 
  (id INT, name STRING, value DOUBLE)

In [0]:
%sql
INSERT INTO students VALUES (1, "Yve", 1.0);
INSERT INTO students VALUES (2, "Omar", 2.5);
INSERT INTO students VALUES (3, "Elia", 3.3);

In [0]:
%sql
SELECT * FROM students

In [0]:
%sql
INSERT INTO students
VALUES 
  (4, "Ted", 4.7),
  (5, "Tiffany", 5.5),
  (6, "Vini", 6.3)

In [0]:
%sql
SELECT * FROM students

In [0]:
%sql
UPDATE students 
SET value = value + 1
WHERE name LIKE "T%"

In [0]:
%sql
SELECT * FROM students

In [0]:
%python
spark.sql("""
DELETE FROM students 
WHERE value > 6
""")

In [0]:
%sql
SELECT * FROM students

%md



## Using Merge

Some SQL systems have the concept of an upsert, which allows updates, inserts, and other data manipulations to be run as a single command.

Databricks uses the **`MERGE`** keyword to perform this operation.

Consider the following temporary view, which contains 4 records that might be output by a Change Data Capture (CDC) feed.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW updates(id, name, value, type) AS VALUES
  (2, "Omar", 15.2, "update"),
  (3, "", null, "delete"),
  (7, "Blue", 7.7, "insert"),
  (11, "Diya", 8.8, "update");
  
SELECT * FROM updates;

In [0]:
%sql
MERGE INTO students b
USING updates u
ON b.id=u.id
WHEN MATCHED AND u.type = "update"
  THEN UPDATE SET *
WHEN MATCHED AND u.type = "delete"
  THEN DELETE
WHEN NOT MATCHED AND u.type = "insert"
  THEN INSERT *

%md
The query you provided performs a **MERGE operation** to synchronize the data between the `students` table (target) and the `updates` temporary view (source). Here's a detailed explanation of the query:

---

### **Query Breakdown:**

#### **1. `MERGE INTO students b USING updates u ON b.id=u.id`**
   - **What it does**: This merges data from the `updates` view (`u`) into the `students` table (`b`), based on a condition.
   - **Condition**: `ON b.id = u.id`
     - Matches rows in `students` (`b`) and `updates` (`u`) where the `id` values are the same.

#### **2. `WHEN MATCHED` Clauses**
   - These clauses handle cases where a row in `students` has the same `id` as a row in `updates`.

   - **First Clause:**
     ```sql
     WHEN MATCHED AND u.type = "update"
       THEN UPDATE SET *
     ```
     - **Condition**: The rows match by `id`, and the `type` in `updates` is `"update"`.
     - **Action**: Updates all columns (`*`) in `students` with the corresponding values from `updates`.

   - **Second Clause:**
     ```sql
     WHEN MATCHED AND u.type = "delete"
       THEN DELETE
     ```
     - **Condition**: The rows match by `id`, and the `type` in `updates` is `"delete"`.
     - **Action**: Deletes the matching row from `students`.

#### **3. `WHEN NOT MATCHED` Clause**
   - **Condition**: There is no matching `id` in `students` for the `id` in `updates`, and the `type` in `updates` is `"insert"`.
   - **Action**:
     ```sql
     WHEN NOT MATCHED AND u.type = "insert"
       THEN INSERT *
     ```
     - Inserts a new row into `students` with all columns from `updates`.

---

### **Example Execution:**

#### Initial `students` Table:
| id  | name     | value  |
|------|----------|--------|
| 2    | Omar     | 2.5    |
| 3    | Elia     | 3.3    |
| 4    | Ted      | 4.7    |

#### `updates` Temporary View:
| id  | name   | value  | type    |
|-----|--------|--------|---------|
| 2   | Omar   | 15.2   | update  |
| 3   |        | null   | delete  |
| 7   | Blue   | 7.7    | insert  |
| 11  | Diya   | 8.8    | update  |

---

### **Steps and Results After Merge:**

1. **`WHEN MATCHED AND u.type = "update"`**
   - Row with `id = 2` is updated: `value = 15.2`.
   - Row with `id = 11` is skipped as it doesn’t exist in `students`.

2. **`WHEN MATCHED AND u.type = "delete"`**
   - Row with `id = 3` is deleted.

3. **`WHEN NOT MATCHED AND u.type = "insert"`**
   - Row with `id = 7` is inserted.

---

#### Final `students` Table:
| id  | name     | value  |
|-----|----------|--------|
| 2   | Omar     | 15.2   |
| 4   | Ted      | 4.7    |
| 7   | Blue     | 7.7    |

---

### **Key Points:**

- The `MERGE` operation simplifies conditional updates, inserts, and deletions in a single query.
- The use of `SET *` and `INSERT *` updates/inserts all columns directly. This approach assumes column alignment between the target and source.
- It's ideal for syncing data between tables or applying change data capture (CDC) updates.

Let me know if you’d like further examples or explanations! 😊

In [0]:
%sql
DROP TABLE students

In [0]:
CREATE TABLE IF NOT EXISTS beans
 (name	STRING,color	STRING,grams	FLOAT,delicious	BOOLEAN);

In [0]:
SELECT * FROM beans

In [0]:
INSERT INTO beans VALUES
("black", "black", 500, true),
("lentils", "brown", 1000, true),
("jelly", "rainbow", 42.5, false)

In [0]:
SELECT * FROM beans

In [0]:
INSERT INTO beans VALUES
('pinto', 'brown', 1.5, true),
('green', 'green', 178.3, true),
('beanbag chair', 'white', 40000, false)

In [0]:
SELECT * FROM beans

In [0]:
UPDATE beans
SET delicious = true
WHERE name = "jelly"

In [0]:
-- TODO
UPDATE beans
SET grams = 1500
WHERE name = "pinto"


In [0]:
select * from beans

In [0]:
-- TODO
DELETE FROM beans
WHERE delicious = FALSE

In [0]:
CREATE OR REPLACE TEMP VIEW new_beans(name, color, grams, delicious) AS VALUES
('black', 'black', 60.5, true),
('lentils', 'green', 500, true),
('kidney', 'red', 387.2, true),
('castor', 'brown', 25, false);

SELECT * FROM new_beans

Make sure your logic:

Matches beans by name and color
Updates existing beans by adding the new weight to the existing weight
Inserts new beans only if they are delicious

In [0]:
MERGE INTO beans AS T 
USING new_beans AS S ON T.name = S.name
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED AND S.delicious = TRUE THEN INSERT *

In [0]:
select * from beans

In [0]:
UPDATE beans SET name = '' WHERE name = 'green';



In [0]:
select * from beans

### Advanced Delta Lake Features

In [0]:
CREATE TABLE IF NOT EXISTS students
  (id INT, name STRING, value DOUBLE);
  
INSERT INTO students VALUES (1, "Yve", 1.0);
INSERT INTO students VALUES (2, "Omar", 2.5);
INSERT INTO students VALUES (3, "Elia", 3.3);

INSERT INTO students
VALUES 
  (4, "Ted", 4.7),
  (5, "Tiffany", 5.5),
  (6, "Vini", 6.3);
  
UPDATE students 
SET value = value + 1
WHERE name LIKE "T%";

DELETE FROM students 
WHERE value > 6;

CREATE OR REPLACE TEMP VIEW updates(id, name, value, type) AS VALUES
  (2, "Omar", 15.2, "update"),
  (3, "", null, "delete"),
  (7, "Blue", 7.7, "insert"),
  (11, "Diya", 8.8, "update");
  
MERGE INTO students b
USING updates u
ON b.id=u.id
WHEN MATCHED AND u.type = "update"
  THEN UPDATE SET *
WHEN MATCHED AND u.type = "delete"
  THEN DELETE
WHEN NOT MATCHED AND u.type = "insert"
  THEN INSERT *;

In [0]:
DESCRIBE students;

In [0]:
DESCRIBE EXTENDED students;

**`DESCRIBE EXTENDED`** allows us to see important metadata about our table.

**`DESCRIBE DETAIL`** is another command that allows us to explore table metadata.

In [0]:
DESCRIBE DETAIL students;


In [0]:
%python 
dbutils.fs.ls("dbfs:/mnt/training/student")